In [1]:
import os
import numpy as np
import math

import matplotlib.pyplot as plt

import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app

import openmmtools as mmtools
from openmmtools import testsystems

import yank.repex as repex

import tempfile
#from tqdm import tqdm, tqdm_notebook

plt.style.use("ggplot")

# System Definition

In [2]:
testsystem = testsystems.AlanineDipeptideImplicit()

# Parallel Tempering Parameters

In [3]:
n_replicas = 6
T_min      = 298.0*unit.kelvin
T_max      = 600.0*unit.kelvin

temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0) 
                for i in range(n_replicas)]


# Thermodynamic States Definition

In [4]:
thermodynamic_states = [mmtools.states.ThermodynamicState(system=testsystem.system, temperature=T) for T in temperatures]

# REPEX Options

In [5]:
move = mmtools.mcmc.GHMCMove(timestep=2.0*unit.femtoseconds, n_steps=50)
simulation = repex.ReplicaExchange(mcmc_moves=move, number_of_iterations=100)

# Reporters

In [6]:
storage_path = tempfile.NamedTemporaryFile(delete=False).name+'.nc'
print(storage_path)
reporter = repex.Reporter(storage_path, checkpoint_interval=1)

/tmp/tmpgkozobg7.nc


# REPEX Simulation

In [7]:
simulation.create(thermodynamic_states=thermodynamic_states,
                 sampler_states=mmtools.states.SamplerState(testsystem.positions),
                 storage=reporter)

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs sampling: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/1.

In [8]:
simulation.run()

In [9]:
print(simulation.iteration)

100


In [10]:
reporter.close()
del reporter

A simulation can be reloaded from the reported and extended anytime

In [ ]:
# del simulation

In [ ]:
# simulation = repex.ReplicaExchange.from_storage(reporter)

In [ ]:
# simulation.extend(n_iterations=5)

In [ ]:
# simulation.iteration

In [ ]:
# reporter.close()
# del reporter

Info can be extracted even while running

In [11]:
reporter = repex.Reporter(storage=storage_path, open_mode='r', checkpoint_interval=1)

In [13]:
sampler_states = reporter.read_sampler_states(iteration=100)

In [14]:
len(sampler_states)

6

In [16]:
sampler_states[5].positions

Quantity(value=array([[ 0.21991757,  0.16486461, -0.10605509],
       [ 0.29886287,  0.14147374, -0.03463028],
       [ 0.37748697,  0.07437877, -0.06923543],
       [ 0.24978538,  0.10009227,  0.05346043],
       [ 0.37698421,  0.26279372,  0.00101882],
       [ 0.41534773,  0.27282801,  0.11724094],
       [ 0.40525666,  0.35362568, -0.09103757],
       [ 0.36459407,  0.35035881, -0.18343282],
       [ 0.49868962,  0.47166926, -0.07583616],
       [ 0.58496809,  0.43798035, -0.01837235],
       [ 0.54668045,  0.52836931, -0.21680768],
       [ 0.46831369,  0.57188696, -0.27882269],
       [ 0.63141817,  0.59327185, -0.19471295],
       [ 0.5848639 ,  0.44907227, -0.28111175],
       [ 0.44238472,  0.58332765,  0.00587447],
       [ 0.32151783,  0.61412621,  0.00427138],
       [ 0.5326938 ,  0.65151519,  0.07235631],
       [ 0.62195408,  0.6117689 ,  0.09792643],
       [ 0.50154448,  0.7910822 ,  0.11164727],
       [ 0.53775084,  0.87205815,  0.04829909],
       [ 0.39956295,  0.7

In [ ]:
reporter.close()
del reporter

Clean up

In [ ]:
os.remove(storage_path)